In [1]:
%%writefile tcpClient_pr.py
import socket
host=socket.gethostname()
port=5000
client_socket=socket.socket()
client_socket.connect((host,port))
message=input('->')
l=message.split()
n=[]
for i in l:
    n.append(int(i))
odd=list(filter(lambda x:x%2!=0,n))
cube=list(map(lambda x:x**3,odd))
ans=''
for i in cube:
    ans=ans+str(i)+" "
client_socket.send(ans.encode())
client_socket.close()

Overwriting tcpClient_pr.py


In [2]:
%%writefile tcpServer_pr.py
import socket
host=socket.gethostname()
port=5000
server_socket=socket.socket()
server_socket.bind((host,port))
server_socket.listen()
conn,address=server_socket.accept()
data=conn.recv(1024).decode()
print('from connected user:',data)
conn.close()

Overwriting tcpServer_pr.py


In [3]:
%%writefile tcpServer_pr1.py
import socket
host = "localhost"
port = 5000
buffer_size = 1024
with socket.socket() as s:
    s.bind((host,port))
    s.listen()
    conn,addr = s.accept()
    with conn:
        print("connected user :",addr)
        file_data = b""
        while True:
            data = conn.recv(buffer_size)
            if not data:
                break
            file_data = data
        with open('recv.ipynb','wb') as f:
            f.write(file_data)
        print('file received')

Overwriting tcpServer_pr1.py


In [4]:
%%writefile tcpClient_pr1.py
import socket
host = 'localhost'
port = 5000
buffer_size = 1024
with socket.socket() as s:
    s.connect((host,port))
    with open('ch-8.ipynb','rb') as f:
        while True:
            data = f.read(buffer_size)
            if not data:
                break
            s.sendAll(data)
        print('file sent')

Overwriting tcpClient_pr1.py


In [6]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

with open("vishal_new.html", "r", encoding="utf-8") as file:
    html_content = file.read()
    soup = BeautifulSoup(html_content)
    soup = soup.find_all("div", class_="_2kHMtA")
    len(soup)
#   print(soup)

    name_mobile = []
    rating = []
    fprize_mobile = []
    prize_mobile = []
    for i in soup:
        name_mobile.append(i.find("div", class_="_4rR01T").text)
        rating.append(float(i.find("div", class_="_3LWZlK").text))
        fprize_mobile.append(int(i.find("div", class_="_30jeq3 _1_WHN1").text[1:3] + i.find("div", class_="_30jeq3 _1_WHN1").text[4:]))
        try:
            prize_mobile.append(int(i.find("div", class_="_3I9_wc _27UcVY").text[1:].replace(",", "")))
        except:
            prize_mobile.append(np.nan)
    d = {"name_mobile": name_mobile, "rating": rating,"fprize_mobile": fprize_mobile, "prize_mobile": prize_mobile}
    df = pd.DataFrame(d)
    print(df)

                                          name_mobile  rating  fprize_mobile  \
0                    Huawei P9 (Mystic Silver, 32 GB)     4.5          39999   
1                    Huawei P9 (Prestige Gold, 32 GB)     4.5          39999   
2   SAMSUNG Galaxy S22 Plus 5G (Phantom Black, 128...     4.5          49999   
3   SAMSUNG Galaxy Z Flip3 5G (Phantom Black, 128 GB)     4.3          44999   
4   MOTOROLA Edge 30 Ultra (Interstellar Black, 25...     4.2          49999   
5    MOTOROLA Edge 30 Ultra (Starlight White, 256 GB)     4.2          49999   
6      SAMSUNG Galaxy A54 5G (Awesome Violet, 256 GB)     4.3          40999   
7                   OnePlus 8 (Glacial Green, 256 GB)     4.5          46999   
8          SAMSUNG Galaxy S22 Plus 5G (Green, 128 GB)     4.5          49999   
9               OnePlus 8 (Interstellar Glow, 256 GB)     4.5          47349   
10                  Mi 10T Pro (Cosmic Black, 128 GB)     4.2          38849   
11          SAMSUNG Galaxy Z Flip3 5G (C

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score

df = df.dropna()
x = df[["fprize_mobile","rating"]]
y = df["prize_mobile"]

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)
lr = LinearRegression()
lr.fit(x_train,y_train)
y_pred = lr.predict(x_test)
print(lr.coef_ )
print(lr.intercept_)

print(mean_squared_error(y_test, y_pred))
print(r2_score(y_test,y_pred))

[ 2.67628936e+00 -4.91261598e+04]
158446.25133065082
226610141.93806204
0.11614205640107444


In [31]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

website = "https://github.com/topics"
result = requests.get(website)
content = result.text
soup = BeautifulSoup(content,'html.parser')
# print(soup.prettify())
box = soup.findAll('div',class_='py-4 border-bottom d-flex flex-justify-between')
# print(box)

title = []
description = []
url = []
for i in box:
    title.append(i.find('p',class_='f3 lh-condensed mb-0 mt-1 Link--primary').get_text())
    description.append(i.find('p',class_='f5 color-fg-muted mb-0 mt-1').get_text())
    url.append(i.find('a',class_='no-underline flex-1 d-flex flex-column').get_text())
d = {'Title':title,'Description':description,'Url':url}
df = pd.DataFrame(d)
print(df)
df.to_csv('topic.csv',index=False)

                     Title                                        Description  \
0                       3D  \n          3D refers to the use of three-dime...   
1                     Ajax  \n          Ajax is a technique for creating i...   
2                Algorithm  \n          Algorithms are self-contained sequ...   
3                      Amp  \n          Amp is a non-blocking concurrency ...   
4                  Android  \n          Android is an operating system bui...   
5                  Angular  \n          Angular is an open source web appl...   
6                  Ansible  \n          Ansible is a simple and powerful a...   
7                      API  \n          An API (Application Programming In...   
8                  Arduino  \n          Arduino is an open source platform...   
9                  ASP.NET  \n          ASP.NET is a web framework for bui...   
10           Awesome Lists  \n          An awesome list is a list of aweso...   
11     Amazon Web Services  

In [32]:
website = "https://github.com/topics/3d"
result = requests.get(website)
content = result.text
soup = BeautifulSoup(content,'html.parser')
# print(soup.prettify())
box = soup.findAll('article',class_='border rounded color-shadow-small color-bg-subtle my-4')
# print(box)

topic_title = []
username = []
stars = []
topic_url = []
for i in box:
    username.append(i.find('h3',class_='f3 color-fg-muted text-normal lh-condensed').text.split()[0])
    topic_title.append(i.find('h3',class_='f3 color-fg-muted text-normal lh-condensed').text.split()[-1])
    stars.append(i.find('span',class_='Counter js-social-count').get_text())
    topic_url.append("https://github.com/topics/3d"+i.find('h3',class_='f3 color-fg-muted text-normal lh-condensed').get_text())
d = {'Username':username,'Topic_Title':topic_title,'Stars':stars,'Url':topic_url}
df = pd.DataFrame(d)
print(df)
df.to_csv('Topics.csv',index=False)

              Username                    Topic_Title  Stars  \
0               mrdoob                       three.js   106k   
1               pmndrs              react-three-fiber    29k   
2              FreeCAD                        FreeCAD  24.9k   
3            BabylonJS                     Babylon.js  24.2k   
4               libgdx                         libgdx  24.1k   
5                ssloy                   tinyrenderer    22k   
6              lettier  3d-game-shaders-for-beginners  18.8k   
7             aframevr                         aframe  17.1k   
8              blender                        blender  15.3k   
9                 4ian                       GDevelop  14.5k   
10            CesiumGS                         cesium  13.7k   
11             isl-org                         Open3D  12.4k   
12            MonoGame                       MonoGame  12.2k   
13           DavidHDev                     react-bits  11.9k   
14              mapbox                  

In [33]:
def scrape_github_topic(topic):
    url = f"https://github.com/topics/{topic}"
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'html.parser')
    
    # Find all repo containers
    repos = soup.find_all('article', class_='border rounded color-shadow-small color-bg-subtle my-4')
    
    usernames = []
    repo_names = []
    stars = []
    repo_urls = []
    
    for repo in repos:
        h3_tag = repo.find('h3', class_='f3 color-fg-muted text-normal lh-condensed')
        if h3_tag:
            full_name = h3_tag.text.strip().replace('\n', '').replace(' ', '')
            if '/' in full_name:
                username, repo_name = full_name.split('/')
            else:
                parts = full_name.split()
                username = parts[0]
                repo_name = parts[-1]
            
            usernames.append(username)
            repo_names.append(repo_name)
        
            star_tag = repo.find('span', class_='Counter js-social-count')
            star_count = star_tag.text.strip() if star_tag else '0'
            stars.append(star_count)
            
            repo_url = f"https://github.com/{username}/{repo_name}"
            repo_urls.append(repo_url)
    
    df = pd.DataFrame({
        'username': usernames,
        'repo_name': repo_names,
        'stars': stars,
        'repo_url': repo_urls
    })
    csv_filename = f"{topic}_repos.csv"
    df.to_csv(csv_filename, index=False)
    print(f"Saved data to {csv_filename}")

# Example usage
for i in title:
    scrape_github_topic(i)

Saved data to 3D_repos.csv
Saved data to Ajax_repos.csv
Saved data to Algorithm_repos.csv
Saved data to Amp_repos.csv
Saved data to Android_repos.csv
Saved data to Angular_repos.csv
Saved data to Ansible_repos.csv
Saved data to API_repos.csv
Saved data to Arduino_repos.csv
Saved data to ASP.NET_repos.csv
Saved data to Awesome Lists_repos.csv
Saved data to Amazon Web Services_repos.csv
Saved data to Azure_repos.csv
Saved data to Babel_repos.csv
Saved data to Bash_repos.csv
Saved data to Bitcoin_repos.csv
Saved data to Bootstrap_repos.csv
Saved data to Bot_repos.csv
Saved data to C_repos.csv
Saved data to Chrome_repos.csv
Saved data to Chrome extension_repos.csv
Saved data to Command-line interface_repos.csv
Saved data to Clojure_repos.csv
Saved data to Code quality_repos.csv
Saved data to Code review_repos.csv
Saved data to Compiler_repos.csv
Saved data to Continuous integration_repos.csv
Saved data to C++_repos.csv
Saved data to Cryptocurrency_repos.csv
Saved data to Crystal_repos.csv


In [42]:
with open("indeed jobs.html", "r", encoding="utf-8") as file:
    html_content = file.read()
    soup = BeautifulSoup(html_content)
    box = soup.find_all("td", class_="resultContent css-1o6lhys eu4oa1w0")
#     print(box)

    company_name = []
    job_role = []
    city = []
    pay = []
    for i in box:
        company_name.append(i.find('span',class_='css-1h7lukg eu4oa1w0').get_text())
        city.append(i.find('div',class_='css-1restlb eu4oa1w0').get_text())
        job_role.append(i.find('a',class_='jcs-JobTitle css-1baag51 eu4oa1w0').get_text())
        pay.append(i.find('div',class_='css-18z4q2i eu4oa1w0'))
    d = {'company_name':company_name,'job_role':job_role,'city':city,'pay':pay}
    df = pd.DataFrame(d)
    print(df)

                         company_name                         job_role  \
0       Saltriver Infosystems Pvt Ltd          Junior Python Developer   
1                   Techify Solutions                AI & ML - Trainee   
2                             XcelTec         Python Developer Fresher   
3             Arrow Electronics, Inc.           AI ML Python Developer   
4                 Vrinsoft Technology                    Trainee AI ML   
5                 Taasha Technologies                 Python Developer   
6             PySquad Informatics LLP                      AI Engineer   
7                        JuniGadi.com      Full Stack Developer Intern   
8                            Anblicks                     Data Analyst   
9   Sufalam Technologies Private Ltd.                 Python Developer   
10                        Zydus Group  AI & ML Technologies Specialist   
11    eSparkBiz Technologies Pvt. Ltd                   AI/ML Engineer   
12               reverseBits Tech LLP 

In [32]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

with open("newflipkart.html", "r", encoding="utf-8") as file:
    html_content = file.read()
    soup = BeautifulSoup(html_content)
    box = soup.find_all("div", class_="yKfJKb row")
#     print(box)

    name = []
    memory = []
    color = []
    stars = []
    reviews = []
    old_price = []
    new_price = []
    discount = []
    
    for i in box:
        name.append(i.find('div',class_='KzDlHZ').text.split('(')[0])
        memory.append(i.find('li',class_='J+igdf').text.split()[0])
        color.append(i.find('div',class_='KzDlHZ').text.split('(')[1].split(',')[0])
        stars.append(i.find('div',class_='XQDdHH').get_text())
        reviews.append(i.find('span',class_='Wphh3N').text.split()[-2])
        old_price.append(i.find('div',class_='yRaY8j ZYYwLA').get_text())
        new_price.append(i.find('div',class_='Nx9bqj _4b5DiR').get_text())
        discount.append(i.find('div',class_='UkUFwK').get_text())
        
    d = {'Name':name,'Memory':memory,'Color':color,'Stars':stars,'Reviews':reviews,'Old_price':old_price,'New_price':new_price,'Discount':discount}
    df = pd.DataFrame(d)
    print(df)
    df.to_csv('exam.csv',index=False)

                     Name Memory            Color Stars Reviews  Old_price  \
0        Apple iPhone 16     128            Black   4.6     704    ₹79,900   
1        Apple iPhone 16     128             Pink   4.6     704    ₹79,900   
2        Apple iPhone 13     128        Starlight   4.6  13,902    ₹49,900   
3        Apple iPhone 16     128            White   4.6     704    ₹79,900   
4   Apple iPhone 16 Plus     128            White   4.6     143    ₹89,900   
5       Apple iPhone 16e     128            White   4.6      51    ₹59,900   
6        Apple iPhone 16     128             Teal   4.6     704    ₹79,900   
7   Apple iPhone 16 Plus     128            Black   4.6     143    ₹89,900   
8        Apple iPhone 13     128         Midnight   4.6  13,902    ₹49,900   
9        Apple iPhone 13     128             Blue   4.6  13,902    ₹49,900   
10       Apple iPhone 16     128      Ultramarine   4.6     704    ₹79,900   
11       Apple iPhone 15     128             Blue   4.6   9,296 

In [83]:
with open("TrumpLies.html", "r", encoding="utf-8") as file:
    html_content = file.read()
    soup = BeautifulSoup(html_content)
    box = soup.find_all("article", class_="story theme-interactive theme-main")
#     print(box)

    title = ""
    p = []
    h1 = []
    h2 = []
    ll = []
    date = []
    statement = []
    true_text = []
    
    h11 = soup.findAll('h1')
    h21 = soup.findAll('h2')
    l1 = soup.findAll('a')
    total_r = soup.findAll('span','short-desc')

    for _ in h11:
        h1.append(_.text.replace(" ","").replace("\n","").strip())
    for _ in h21:
        h2.append(_.text.strip())
    for _ in l1:
        if(_['href'].startswith('http') or _['href'].startswith('https')):
            ll.append(_['href'])
    for r in total_r:
        d=r.find('strong').get_text(strip=True)
        s=r.get_text(strip=True).split('“')[1].split('”')[0].replace('"','')
        t=r.find('span',class_='short-truth').get_text(strip=True)[1:].replace(')','')  
        date.append(d)
        statement.append(s)
        true_text.append(t)
        
    for i in box:
        a = i.find('h1',class_='custom-hed story-heading interactive-headline').get_text()
        for j in a:
            title = title+j.strip()
        date_footer = i.find('footer',class_='m-statement__footer').text.split('😊')[1]
        p.append(i.find('p',class_='secret-word').get_text())
        
    print(title)
    print(date_footer)
    print(p)
    print(h1)
    print(h2)
    print(ll)
    print("Total Records : ",len(total_r))
    
    d = {'Date':date,'Statement':statement,'True_Text':true_text}
    df = pd.DataFrame(d)
    print(df)

Trump’sLies
August 15, 1947
  

['blackpink']
['PresidentTrump’sLies,theDefinitiveList', 'Trump’sLies']
['Opinion', 'Site Information Navigation']
['http://www.nytimes.com/by/david-leonhardt', 'https://www.nytimes.com/by/stuart-a-thompson', 'https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the', 'http://nation.time.com/2013/11/06/10-things-you-didnt-know-about-time/', 'https://www.nytimes.com/2017/01/23/us/politics/donald-trump-congress-democrats.html', 'https://www.nytimes.com/2017/01/21/us/politics/trump-white-house-briefing-inauguration-crowd-size.html', 'https://www.nytimes.com/2017/01/24/us/politics/unauthorized-immigrant-voting-trump-lie.html', 'https://www.nytimes.com/2017/03/13/us/politics/fact-check-trump-obamacare-health-care.html', 'https://www.dnainfo.com/chicago/2017-chicago-murders/timeline?mon=1', 'https://www.nytimes.com/interactive/2017/01/29/us/refugee-vetting-process.html', 'https://www.washingtonpost.com/news/fact-chec